In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install pandas
!pip install matplotlib

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
driver.get("https://apps.douglas.co.us/assessor/web#/")

# Wait for the search box to be visible and locate it
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'app-input-debounce input[type="text"]'))
)

# Send search query
search_box.send_keys("2719 Castle Glen Dr")
search_box.send_keys(Keys.RETURN)

# Wait for the results to load (you may need to adjust this depending on the page)
driver.implicitly_wait(5)

# Wait until the table rows are loaded (anchor tags with 'table-row' class)
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.table-row'))
)

# Locate the first row (anchor tag with class 'table-row') and click it
first_row = driver.find_element(By.CSS_SELECTOR, 'a.table-row')
first_row.click()

# Get page source to parse the HTML after clicking the first row
page_source = driver.page_source

# BeautifulSoup to parse the HTML for further scraping
soup = BeautifulSoup(page_source, 'html.parser')

### Closes the pop-up that appears

In [3]:
# Wait for the pop-up dialog to appear and then click the "Close" button
wait = WebDriverWait(driver, 10)

# Wait for the "Close" button to be clickable
close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button/span[text()='Close']")))

# Click the "Close" button to dismiss the pop-up
close_button.click()

# Wait for the account summary section to be loaded
wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='dropdown-content']")))

# Extract HTML content
html_content = driver.page_source

# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

## Account Summary
* TODO: Section PDF Map Link

In [4]:
# Extract Toggle Button and Links before Account Summary
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

# Initialize a dictionary to store toggle button and links
key_value_pairs = {}

# Extract the toggle button text (key) and status (value)
toggle_button = soup.find('span', class_='ui-button-text')
if toggle_button:
    key_value_pairs["Toggle Button"] = toggle_button.text.strip()

# Extract the anchor tags (links) and their href attributes
links = soup.find_all('a', href=True)
for link in links:
    link_text = link.get_text(strip=True)
    link_url = link['href']
    key_value_pairs[link_text] = f'<a href="{link_url}">{link_text}</a>'

# Now proceed with the Account Summary logic
# Target the dropdown for Account Summary by using its ID
dropdown_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@id='SummaryAccountInfo']//span[@class='bar faux-button']")))

# Click the dropdown to expand it
dropdown_button.click()

# Wait for the Account Summary content to load
wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='SummaryAccountInfo']//div[@class='dropdown-content']")))

# Extract HTML content again after the dropdown is expanded
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the dropdown content specifically under the 'SummaryAccountInfo' ID
dropdown_content = soup.find('div', id='SummaryAccountInfo').find('div', class_='dropdown-content')

# Extract the key-value pairs from Account Summary
account_summary_pairs = {}

# Find all the rows with the class 'skinny-row' which contain the label and value pairs
rows = dropdown_content.find_all('div', class_='skinny-row')

for row in rows:
    # Extract the label (key) and value (value)
    label = row.find('div', class_='col-xs-4').text.strip() if row.find('div', class_='col-xs-4') else None
    value = row.find('div', class_='col-xs-8').text.strip() if row.find('div', class_='col-xs-8') else None
    
    # Add to dictionary if both label and value exist
    if label and value:
        account_summary_pairs[label] = value

# Print the Account Summary key-value pairs
print(f"Account Summary:")
for key, value in account_summary_pairs.items():
    # Skip the "Update Mailing Address" entry
    if key != 'Update Mailing Address':
        print(f"{key}: {value}")

# Extract additional data (Location Description, Owner Info, PLSS Location)
location_description = soup.find('div', string='Location Description').find_next('div').text.strip()

# For Owner Info, we need to extract and clean it
owner_info_div = soup.find('div', string='Owner Info').find_next('div')

# Extract owner name and address
owner_info_raw = owner_info_div.text.strip()

# Split owner info into lines
owner_info_parts = owner_info_raw.split("\n")

# Clean up and extract the name and address properly
owner_name = owner_info_parts[0].strip() 
owner_address = " ".join(owner_info_parts[1:]).strip()  

# If "Update Mailing Address" appears in the address, remove it
if "Update Mailing Address" in owner_address:
    owner_address = owner_address.split("Update Mailing Address")[0].strip()

# Extract PLSS Location
plss_location = soup.find('div', string='Public Land Survey System (PLSS) Location').find_next('div').text.strip()

# Clean the PLSS Location
plss_location_cleaned = ' '.join(plss_location.split())

# Optionally, reformat for better readability (if you want to format it neatly)
plss_location_cleaned = plss_location_cleaned.replace("Quarter:", "\nQuarter:").replace("Section:", "\nSection:").replace("Township:", "\nTownship:").replace("Range:", "\nRange:")

# Extract the Section PDF Map link (if it exists)
section_pdf_map = None

# Find all the div elements with class 'skinny-row'
pdf_map_rows = soup.find_all('div', class_='skinny-row')

for row in pdf_map_rows:
    # Look for an anchor tag within the row
    link = row.find('a', href=True)
    if link and "SectionMap" in link['href']:  # Check if the href contains "SectionMap"
        section_pdf_map = link['href']
        break 

# Print other extracted information
print(f"\nLocation Description: {location_description}")
print(f"\nOwner Info:")
print(f"Owner Name: {owner_name}")
print(f"Owner Address: {owner_address}")
print(f"\nPublic Land Survey System (PLSS) Location: {plss_location_cleaned}")
print(f"\nSection PDF Map Link: {section_pdf_map}")

# Close the browser
# driver.quit()


Account Summary:
Account #:: R0396965
State Parcel #:: 2351-163-01-115
Account Type:: Residential
Tax District:: 3473
Neighborhood-Ext:: 410-I
Building Count:: 1
Building Permit Authority:: Douglas County (website )
Phone:: 303-660-7497
Name:: CASTLE PINES
Reception No:: 9607889

Location Description: LOT 19A CASTLE PINES # 1A 16TH AMENDMENT    0.23 AM/L

Owner Info:
Owner Name: DENNIS R HENDRIX 2020 COLORADO PERSONAL RESIDENCE TRUST & JENNIE M HENDRIX 2020 COLORADO PERSONAL RESIDENCE TRUST
Owner Address: 2719 CASTLE GLEN DRCASTLE ROCK, CO 80108

Public Land Survey System (PLSS) Location: 
Quarter: SW; 
Section: 16; 
Township: 7; 
Range: 67

Section PDF Map Link: /realware/SectionMaps/TWP2351/DC_2351_16.pdf


## Valuation Information
* TODO: Show graphs


In [5]:
# Initialize a dictionary to store toggle button and links
key_value_pairs = {}

# Extract the toggle button text (Show Graphs)
toggle_button = soup.find('span', class_='ui-button-text')
if toggle_button:
    print(toggle_button.text.strip())  # Print 'Show Graphs'

# Extract the anchor tags (links) and their href attributes
links = soup.find_all('a', href=True)
for link in links:
    link_text = link.get_text(strip=True)
    link_url = link['href']
    
    # Print specific links that we are interested in
    if link_text == "Get Taxes Due":
        print(f'{link_text}: {link_url}')  # Hyperlink Get Taxes Due
    elif link_text == "Property Tax Calculation":
        print(f'{link_text}: {link_url}')  # Hyperlink Property Tax Calculation

# Now continue with the part for finding and processing the table data
# Find the table with class 'value-data-table'
table = soup.find('table', class_='value-data-table')
if table:
    # print("Table found")  

    # Find all tbody elements inside the table with the 'sales-info' class, without specifying the dynamic part
    rows = table.find_all('tbody', class_='value-row')  

    # Check how many rows are found
    # print(f"Found {len(rows)} tbody elements.")  

    # Iterate over each row and extract data
    sales_data = []
    for row in rows:
        # Extract Year, Actual Value, Assessed Value, Tax Rate, Est. Tax Amount
        year = row.find_all('td')[0].text.strip() if len(row.find_all('td')) > 1 else None
        actual_value = row.find_all('td')[1].text.strip() if len(row.find_all('td')) > 1 else None
        assesssed_value = row.find_all('td')[2].text.strip() if len(row.find_all('td')) > 1 else None
        tax_rate = row.find_all('td')[3].text.strip() if len(row.find_all('td')) > 1 else None
        est_tax_amount = row.find_all('td')[4].text.strip() if len(row.find_all('td')) > 1 else None
    
        # Store row data in a dictionary
        row_data = {
            'Year': year,
            'Actual Value': actual_value,
            'Assessed Value': assesssed_value,
            'Tax Rate': tax_rate,
            'Est. Tax Amount': est_tax_amount
        }
        
        sales_data.append(row_data)

    # Print extracted data
    for row in sales_data:
        print("\n")
        for key, value in row.items():
            print(f"{key}: {value}")

else:
    print("Table not found.")

Show graphs
Get Taxes Due: http://apps.douglas.co.us/treasurer/treasurerweb/account.jsp?account=R0396965&guest=true
Property Tax Calculation: https://www.douglas.co.us/assessor/residential-property-tax-calculations


Year: 2024
Actual Value: $1,502,829
Assessed Value: $100,690
Tax Rate: 11.6982%
Est. Tax Amount: Tax Calculation


Year: 2023
Actual Value: $1,502,829
Assessed Value: $100,690
Tax Rate: 11.7909%
Est. Tax Amount: Tax Calculation


Year: 2022
Actual Value: $1,082,358
Assessed Value: $75,220
Tax Rate: 11.4115%
Est. Tax Amount: $8,584


Year: 2021
Actual Value: $1,082,358
Assessed Value: $77,390
Tax Rate: 11.5119%
Est. Tax Amount: $8,909


Year: 2020
Actual Value: $1,263,238
Assessed Value: $90,320
Tax Rate: 11.5502%
Est. Tax Amount: $10,432


Year: 2019
Actual Value: $1,263,238
Assessed Value: $90,320
Tax Rate: 11.5845%
Est. Tax Amount: $10,463


Year: 2018
Actual Value: $1,373,919
Assessed Value: $98,920
Tax Rate: 11.7284%
Est. Tax Amount: $11,602


Year: 2017
Actual Value: 

## Sales History
* Show graphs

In [6]:
# Find the table with class 'sales-data-table table'
table = soup.find('table', class_='sales-data-table table')
# Extract the anchor tags (links) and their href attributes

links = soup.find_all('a', href=True)
# Use a set to track printed links, avoid duplications
printed_links = set()  

for link in links:
    link_text = link.get_text(strip=True)
    link_url = link['href']
    
    # Print specific links that we are interested in, only if not already printed
    if link_text == "View Neighborhood Sales" and link_url not in printed_links:
        print(f'{link_text}: {link_url}')
        printed_links.add(link_url)  # Mark this link as printed
    elif link_text == "Recorded Document Search" and link_url not in printed_links:
        print(f'{link_text}: {link_url}')
        printed_links.add(link_url)

        
if table:
    # print("Table found")  
    
    # Find all tbody elements inside the table with the 'sales-info' class, without specifying the dynamic part
    rows = table.find_all('tbody', class_='sales-info')  
    # print(f"Found {len(rows)} tbody elements.")  
    
    # Iterate over each row and extract data
    sales_data = []
    for row in rows:
        # Extract Reception No, Sale Date, Sale Price, Deed Type, etc.
        reception_no = row.find_all('td')[0].text.strip() if len(row.find_all('td')) > 0 else None
        
        sale_date = row.find_all('td')[1].text.strip() if len(row.find_all('td')) > 1 else None
        sale_price = row.find_all('td')[2].text.strip() if len(row.find_all('td')) > 1 else None
        deed_type = row.find_all('td')[3].text.strip() if len(row.find_all('td')) > 2 else None
        
        # Extract Grantor and Grantee
        sales_details_row = row.find_next('tr', class_='sales-details')
        if sales_details_row:
            grantor_grantee_div = sales_details_row.find('div', class_='col-sm-9 col-xs-12')
            if grantor_grantee_div:
                grantor = grantor_grantee_div.find_all('span', class_='ng-star-inserted')[0].text.strip().replace('Grantor:', '').strip()
                grantee = grantor_grantee_div.find_all('span', class_='ng-star-inserted')[1].text.strip().replace('Grantee:', '').strip()
            else:
                grantor, grantee = None, None
        else:
            grantor, grantee = None, None
        
        # Store row data in a dictionary
        row_data = {
            'Reception No': reception_no,
            'Sale Date': sale_date,
            'Sale Price': sale_price,
            'Deed Type': deed_type,
            'Grantor': grantor,
            'Grantee': grantee
        }
        
        sales_data.append(row_data)

    # Print extracted data
    for row in sales_data:
        print("\n")
        for key, value in row.items():
            print(f"{key}: {value}")
        

else:
    print("Table not found.")

View Neighborhood Sales: https://co-douglas-residential.comper.info/template.aspx?propertyID=R0396965
Recorded Document Search: https://apps.douglas.co.us/LandMarkWeb


Reception No: 2020118714
Sale Date: 11/30/2020
Sale Price: $0
Deed Type: Bargain & Sale Common
Grantor: HENDRIX 2006 FAMILY TRUST
Grantee: DENNIS R HENDRIX & JENNIE M HENDRIX


Reception No: 2020118716
Sale Date: 11/30/2020
Sale Price: $0
Deed Type: Bargain & Sale
Grantor: DENNIS R HENDRIX
Grantee: DENNIS R HENDRIX 2020 COLORADO PERSONAL RESIDENCE TRUST


Reception No: 2020118717
Sale Date: 11/30/2020
Sale Price: $0
Deed Type: Bargain & Sale
Grantor: JENNIE M HENDRIX
Grantee: JENNIE M HENDRIX 2020 COLORADO PERSONAL RESIDENCE TRUST


Reception No: 2018058752
Sale Date: 09/25/2018
Sale Price: $1,290,000
Deed Type: Warranty Deed
Grantor: BRIAN J HUBBELL & SUSAN A HUBBELL
Grantee: HENDRIX 2006 FAMILY TRUST


Reception No: 2015045662
Sale Date: 06/30/2015
Sale Price: $1,300,000
Deed Type: Warranty Deed Joint
Grantor: SIMONS 

## Building Details

* <span style="color: red;">having trouble formatting Primary Info section</span>

* picture (x2)


In [7]:
building_data = {}

# Find the building details section using its ID
building_details = soup.find('div', {'id': 'BuildingDetails'})

if building_details:
    # Scrape building images
    images = building_details.find_all('img', class_='bordered')
    image_urls = [img['src'] for img in images if img.has_attr('src')]
    building_data['Images'] = image_urls
    
    # Scrape building primary info (property type, year built, etc.)
    building_info = building_details.find_all('div', class_='smart-table')
    primary_info = {}

    for info in building_info:
        # Find the label and value pairs for each group
        label_elements = info.find_all('div', recursive=False)
        
        # Ensure there are exactly two divs, one for the label and one for the value
        if len(label_elements) == 2:
            label = label_elements[0].text.strip().replace('\n', '').replace('\r', '')
            value = label_elements[1].text.strip().replace('\n', '').replace('\r', '')
            
            # Add the pair to the primary info dictionary
            primary_info[label] = value

    building_data['\nPrimary Info'] = primary_info

    
    # Scrape additional features and fixtures
    additional_features = []
    more_details = building_details.find_all('div', class_='skinny-row')
    for detail in more_details:
        name = detail.find('span', class_='name')
        value = detail.find('span', class_='value')
        if name and value:
            name_text = name.text.strip().replace('\n', ' ')
            value_text = value.text.strip().replace('\n', ' ')
            additional_features.append({name_text: value_text})
    
    building_data['\nAdditional Features'] = additional_features

else:
    print("Building details section not found.")

# Print the scraped data, each key-value pair on its own line
for key, value in building_data.items():
    print(f"{key}:")
    if isinstance(value, list):
        for item in value:
            # If the value is a dictionary (for additional features)
            if isinstance(item, dict): 
                for sub_key, sub_value in item.items():
                    print(f"  {sub_key} {sub_value}")
            else:
                print(f"  {item}")
    else:
        # For primary info, print each label-value pair on a new line
        if key == '\nPrimary Info': 
            for label, value in value.items():
                print(f"  {label} {value}")
        else:
            print(f"  {value}")

# Close the browser after scraping
driver.quit()

Images:
  /realware/PHOTOS/R0396965/2009_IMP_1_WEBafront.jpg
  /realware/SKETCHES/R0396965/2023_IMP_1_Page 1 - Apex R0396965 Imp No - 1.00.JPG

Primary Info:
  Property Type:                                  Residential Year Built:1997
  Square Footage:                                  3,463 sqft Style:Ranch 1 Story

Additional Features:
  Quality: Excellent
  Quality: Excellent
  % Complete: 100%
  Stories: 1
  Bedrooms (above ground): 1
  Bathrooms (above ground): 2
  Basement Area: 3,379 sqft
  Finished Bsmt. Area: 2,710 sqft (80%)
  Total Finished Area: 6,173 sqft
  Walkout: Y
  Fireplaces: 3
  Porch/Deck Area: 1,384 sqft
  Garage Type Garage Area
  Attached: 1,078 sqft
  Detached: 0 sqft
  Assessor's Building ID: 1


## Land Details

In [8]:
land_info = {}

# Find the "LandInfoAndValue" section
land_info_section = soup.find('div', {'id': 'LandInfoAndValue'})

if land_info_section:
    # Scrape Land Details (Land Type, Class Code, etc.)
    land_details = land_info_section.find_all('div', class_='row')
    
    for detail in land_details:
        label = detail.find('div', class_='col-xs-3')
        value = detail.find('div', class_='col-xs-9')
        
        if label and value:
            # Clean up label and value text
            label_text = label.text.strip().replace('\n', '').replace('\r', '')
            value_text = value.text.strip().replace('\n', '').replace('\r', '')
            
            # Add label-value pair to the dictionary
            land_info[label_text] = value_text
    
    # Scrape Land Valuation (Actual Value)
    valuation_section = land_info_section.find('div', class_='header')
    if valuation_section and 'Land Valuation' in valuation_section.text:
        # Last row is the valuation row
        valuation_row = land_info_section.find_all('div', class_='row')[-1]  
        actual_value_label = valuation_row.find('div', class_='col-sm-3')
        actual_value = valuation_row.find('div', class_='col-sm-9')
        
        if actual_value_label and actual_value:
            land_info['Actual Value'] = actual_value.text.strip().replace('\n', '').replace('\r', '')

# Print the scraped land info
for key, value in land_info.items():
    print(f"{key} {value}")

Land Type: Residential
Class Code: 1112
Class Code Descr.: IMPROVED RESIDENTIAL LAND
Acreage: 0.230 acres
LEA Code: 4AA
Actual Value $350,855


## Tax Authorities
* Show graphs

In [9]:
# Find the table with class 'tax-data-table table'
table = soup.find('table', class_='tax-data-table table')

if table:
    # Find all tbody elements inside the table with the 'tax-info' class (except the last 'total-row')
    rows = table.find_all('tbody', class_='tax-info')  
    
    sales_data = []
    for row in rows:
        # Extract data from the first row
        tax_id = row.find_all('td')[0].text.strip() if len(row.find_all('td')) > 0 else None
        authority_name = row.find_all('td')[1].text.strip() if len(row.find_all('td')) > 1 else None
        mills = row.find_all('td')[2].text.strip() if len(row.find_all('td')) > 2 else None
        tax_rate = row.find_all('td')[3].text.strip() if len(row.find_all('td')) > 3 else None
        tax_amount = row.find_all('td')[4].text.strip() if len(row.find_all('td')) > 4 else None
        
        # Store row data in a dictionary
        row_data = {
            'ID': tax_id,
            'Authority Name': authority_name,
            'Mills': mills,
            'Tax Rate': tax_rate,
            'Est. Tax Amount': tax_amount
        }
        
        sales_data.append(row_data)

    # Print extracted data
    for row in sales_data:
        print("\n")
        for key, value in row.items():
            print(f"{key}: {value}")
        # print('---' * 50)

else:
    print("Table not found.")




ID: 2001
Authority Name: Douglas County Re-1 School District
Mills: 40.324
Tax Rate: 4.0324%
Est. Tax Amount: $4,060


ID: 4016
Authority Name: Castle Pines Metro District
Mills: 33.834
Tax Rate: 3.3834%
Est. Tax Amount: $3,407


ID: 0001
Authority Name: Douglas County Government
Mills: 18.726
Tax Rate: 1.8726%
Est. Tax Amount: $1,886


ID: 4014
Authority Name: South Metro Fire Rescue Fire Protection District
Mills: 9.290
Tax Rate: 0.9290%
Est. Tax Amount: $935


ID: 2004
Authority Name: Douglas County Schools - Debt Service
Mills: 5.204
Tax Rate: 0.5204%
Est. Tax Amount: $524


ID: 0002
Authority Name: Douglas County Law Enforcement
Mills: 4.500
Tax Rate: 0.4500%
Est. Tax Amount: $453


ID: 4390
Authority Name: Douglas Public Library District
Mills: 4.000
Tax Rate: 0.4000%
Est. Tax Amount: $403


ID: 4002
Authority Name: Urban Drainage & Flood Control District
Mills: 0.900
Tax Rate: 0.0900%
Est. Tax Amount: $91


ID: 4012
Authority Name: Cedar Hill Cemetery Association
Mills: 0.104


### Total

In [10]:
all_total_data = []

total_row = table.find_all('tbody')[-1] 

if total_row:
    total_cells = total_row.find_all('td')
    if len(total_cells) >= 5:
        total_data = {
            'Total Authorities': total_cells[1].text.strip(),
            'Total Mills': total_cells[2].text.strip(),
            'Total Tax Rate': total_cells[3].text.strip(),
            'Total Tax Amount': total_cells[4].text.strip()
        }
        all_total_data.append(total_data)

# Print extracted data
for row in all_total_data:
    for key, value in row.items():
        print(f"{key}: {value}")

Total Authorities: 13 Authorities
Total Mills: 116.982
Total Tax Rate: 11.6982%
Total Tax Amount: $11,779


## Documents 
* actual pull pdf

In [11]:
dropdown_content = soup.find('div', id='Documents').find('div', class_='dropdown-content')

# Find the list of documents inside the dropdown
documents_list = dropdown_content.find_all('li', class_='ng-star-inserted') if dropdown_content else []

if documents_list:
    # Initialize an empty dictionary to store document data
    document_data = {}

    # Iterate through each document item and extract the desired data
    for doc in documents_list:
        # Get document name (PDF filename)
        doc_name = doc.find('a').text.strip() if doc.find('a') else None
        
        # Get file size
        size = doc.find('div', class_='col-sm-2')
        size = size.text.strip().replace('Size:', '').strip() if size else None
        
        # Get last modified date
        last_modified = doc.find('div', class_='col-sm-4')
        last_modified = last_modified.text.strip().replace('Last Modified Date:', '').strip() if last_modified else None
        
        # Store the extracted data in the dictionary with document name as the key
        document_data[doc_name] = {
            'Size': size,
            'Last Modified Date': last_modified
        }

    # Print the document data dictionary
    for doc_name, details in document_data.items():
        print(f"\nDocument Name: {doc_name}")
        for key, value in details.items():
            print(f"  {key}: {value}")
        # print('---' * 50)

else:
    print("No documents found ")

# Close the browser
# driver.quit()


Document Name: R0396965_NOV_2024.pdf
  Size: 138.9kb
  Last Modified Date: Apr 23, 2024

Document Name: R0396965_NOV_2023.pdf
  Size: 129.7kb
  Last Modified Date: May 1, 2023

Document Name: R0396965_NOV_2022.pdf
  Size: 120.2kb
  Last Modified Date: Apr 21, 2022

Document Name: R0396965_NOV_2021.pdf
  Size: 322.8kb
  Last Modified Date: Jul 7, 2021

Document Name: R0396965_NOV_2020.pdf
  Size: 324kb
  Last Modified Date: Jun 11, 2020

Document Name: R0396965_NOV_2019.pdf
  Size: 324kb
  Last Modified Date: Jun 15, 2020

Document Name: R0396965_NOV_2018.pdf
  Size: 324.9kb
  Last Modified Date: Jun 17, 2020

Document Name: R0396965_NOV_2017.pdf
  Size: 323.7kb
  Last Modified Date: Jun 18, 2020


## Notifications

In [12]:
dropdown_content = soup.find('div', id='TaxInformation').find('div', class_='dropdown-content')

# Find the list of notifications inside the dropdown (each notification is wrapped in a <div> with the 'ng-star-inserted' class)
notifications_list = dropdown_content.find_all('div', class_='ng-star-inserted') if dropdown_content else []

if notifications_list:
    # Initialize a list to store notification data
    notifications_data = []

    # Iterate through each notification item and extract the desired data
    for notification in notifications_list:
        notification_text = notification.find('span').text.strip() if notification.find('span') else None
        
        # Add each notification text to the list
        notifications_data.append(notification_text)

    # Print the notifications data
    for index, notification in enumerate(notifications_data):
        print(f"\nNotification {notification}")
        # print('---' * 50)
else:
    print("No notifications found in the dropdown.")

# Close the browser
driver.quit()


Notification 1. The 2024 Actual Value for this parcel is reflective of the statutory Appraisal Date of June 30, 2022. Sale transactions that occurred after that date will be considered in the 2025 reassessment.

Notification 2. In compliance with Senate Bill 24-233, the 2024 Taxable Actual Value on this property will be reduced by $55,000, which only applies to certain residential property, and will be reflected on next year’s tax bill.

Notification 3. This property was eligible for a property tax relief payment from Douglas County Government, which are equal to the 2023 Adjusted Assessed Value times the 3.679 Reserve Mill Levy approved by the Board of County Commissioners.
